# Open Channel Flow Calculator - Google Colab Version

This notebook contains a complete open channel flow calculator for hydraulic engineering.

**Features:**
- Normal depth and critical depth calculations (Manning's & Chezy equations)
- Multiple channel geometries (rectangular, trapezoidal, circular, triangular, wide)
- Hydraulic jump analysis
- Gradually varied flow (GVF) profiles
- Weir and sluice gate calculations
- Flow regime classification (subcritical, critical, supercritical)

**Author:** Open Channel Flow Calculator  
**Date:** August 2024

---

## 1. Installation and Setup

First, install required dependencies:

In [ ]:
# Install dependencies (only needed on first run)
!pip install numpy scipy -q

import math
import numpy as np
from scipy.optimize import fsolve, brentq

print("Dependencies installed successfully!")
print(f"NumPy version: {np.__version__}")

## 2. Physical Constants

In [ ]:
# Physical constants
g = 9.81  # Acceleration due to gravity (m/s²)

## 3. Channel Geometry Classes

These classes define different channel cross-sections and provide methods to calculate geometric properties.

In [ ]:
class ChannelSection:
    """Base class for channel cross-sections"""
    
    def area(self, depth):
        """Calculate flow area for given depth"""
        raise NotImplementedError
    
    def wetted_perimeter(self, depth):
        """Calculate wetted perimeter for given depth"""
        raise NotImplementedError
    
    def hydraulic_radius(self, depth):
        """Calculate hydraulic radius R = A/P"""
        A = self.area(depth)
        P = self.wetted_perimeter(depth)
        return A / P if P > 0 else 0
    
    def top_width(self, depth):
        """Calculate top width of water surface"""
        raise NotImplementedError
    
    def hydraulic_depth(self, depth):
        """Calculate hydraulic depth D = A/T"""
        A = self.area(depth)
        T = self.top_width(depth)
        return A / T if T > 0 else 0


class RectangularChannel(ChannelSection):
    """Rectangular channel cross-section"""
    
    def __init__(self, width):
        self.width = width
    
    def area(self, depth):
        return self.width * depth
    
    def wetted_perimeter(self, depth):
        return self.width + 2 * depth
    
    def top_width(self, depth):
        return self.width


class TrapezoidalChannel(ChannelSection):
    """Trapezoidal channel cross-section"""
    
    def __init__(self, bottom_width, side_slope):
        """
        Parameters:
        - bottom_width: bottom width (m)
        - side_slope: side slope (horizontal:vertical), e.g., 1.5 means 1.5H:1V
        """
        self.bottom_width = bottom_width
        self.side_slope = side_slope
    
    def area(self, depth):
        return (self.bottom_width + self.side_slope * depth) * depth
    
    def wetted_perimeter(self, depth):
        return self.bottom_width + 2 * depth * math.sqrt(1 + self.side_slope**2)
    
    def top_width(self, depth):
        return self.bottom_width + 2 * self.side_slope * depth


class CircularChannel(ChannelSection):
    """Circular channel cross-section (pipe)"""
    
    def __init__(self, diameter):
        self.diameter = diameter
        self.radius = diameter / 2
    
    def _theta(self, depth):
        """Calculate central angle from depth"""
        y = depth
        R = self.radius
        if y <= 0:
            return 0
        if y >= 2 * R:
            return math.pi
        cos_theta = 1 - y / R
        cos_theta = max(-1, min(1, cos_theta))
        return math.acos(cos_theta)
    
    def area(self, depth):
        theta = self._theta(depth)
        R = self.radius
        return R**2 * (theta - math.sin(theta) * math.cos(theta))
    
    def wetted_perimeter(self, depth):
        theta = self._theta(depth)
        return 2 * self.radius * theta
    
    def top_width(self, depth):
        theta = self._theta(depth)
        return 2 * self.radius * math.sin(theta)


class TriangularChannel(ChannelSection):
    """V-shaped triangular channel"""
    
    def __init__(self, side_slope=None, semi_angle=None):
        """
        Parameters:
        - side_slope: side slope (horizontal:vertical)
        - semi_angle: semi-angle from vertical in degrees
        """
        if side_slope is not None:
            self.side_slope = side_slope
        elif semi_angle is not None:
            self.side_slope = math.tan(math.radians(semi_angle))
        else:
            raise ValueError("Must provide either side_slope or semi_angle")
    
    def area(self, depth):
        return self.side_slope * depth**2
    
    def wetted_perimeter(self, depth):
        return 2 * depth * math.sqrt(1 + self.side_slope**2)
    
    def top_width(self, depth):
        return 2 * self.side_slope * depth


class WideChannel(RectangularChannel):
    """Wide rectangular channel (unit width analysis)"""
    
    def __init__(self):
        super().__init__(width=1.0)
    
    def hydraulic_radius(self, depth):
        """For wide channels, R ≈ y"""
        return depth


print("Channel geometry classes loaded successfully!")

## 4. Flow Calculations

Functions for calculating normal depth, critical depth, Froude number, and flow regimes.

In [ ]:
def manning_velocity(R, S, n):
    """Calculate velocity using Manning's equation: V = (1/n) * R^(2/3) * S^(1/2)"""
    return (1.0 / n) * (R ** (2/3)) * (S ** 0.5)


def manning_discharge(channel, depth, S, n):
    """Calculate discharge using Manning's equation"""
    A = channel.area(depth)
    R = channel.hydraulic_radius(depth)
    V = manning_velocity(R, S, n)
    return A * V


def normal_depth_manning(channel, Q, S, n, initial_guess=1.0, max_depth=20.0):
    """Calculate normal depth using Manning's equation"""
    def equation(y):
        if y <= 0:
            return 1e10
        Q_calc = manning_discharge(channel, y, S, n)
        return Q_calc - Q
    
    try:
        result = fsolve(equation, initial_guess, full_output=True)
        if result[2] == 1 and result[0][0] > 0:
            return result[0][0]
    except:
        pass
    
    try:
        y_n = brentq(equation, 0.001, max_depth)
        return y_n
    except:
        result = fsolve(equation, max_depth/2)
        return abs(result[0])


def critical_depth(channel, Q, initial_guess=1.0, max_depth=20.0):
    """Calculate critical depth where Froude number = 1"""
    def equation(y):
        if y <= 0:
            return 1e10
        A = channel.area(y)
        T = channel.top_width(y)
        if A <= 0 or T <= 0:
            return 1e10
        return Q**2 * T / (g * A**3) - 1.0
    
    try:
        y_c = brentq(equation, 0.001, max_depth)
        return y_c
    except:
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def froude_number(channel, depth, Q):
    """Calculate Froude number: Fr = V / sqrt(g * D)"""
    A = channel.area(depth)
    if A == 0:
        return 0
    V = Q / A
    D = channel.hydraulic_depth(depth)
    if D == 0:
        return float('inf')
    return V / math.sqrt(g * D)


def flow_regime(Fr):
    """Determine flow regime from Froude number"""
    if Fr < 0.99:
        return 'subcritical'
    elif Fr > 1.01:
        return 'supercritical'
    else:
        return 'critical'


def specific_energy(channel, depth, Q):
    """Calculate specific energy: E = y + V²/(2g)"""
    A = channel.area(depth)
    if A == 0:
        return float('inf')
    V = Q / A
    return depth + V**2 / (2 * g)


def critical_slope_manning(channel, Q, n, y_c=None):
    """Calculate critical slope (slope at which normal depth = critical depth)"""
    if y_c is None:
        y_c = critical_depth(channel, Q)
    
    A = channel.area(y_c)
    R = channel.hydraulic_radius(y_c)
    V = Q / A
    
    S_c = (V * n / (R ** (2/3))) ** 2
    return S_c


def slope_classification(S, S_c):
    """Classify channel slope as mild, critical, or steep"""
    if S < S_c * 0.99:
        return 'mild'
    elif S > S_c * 1.01:
        return 'steep'
    else:
        return 'critical'


print("Flow calculation functions loaded successfully!")

## 5. Hydraulic Jump Analysis

Functions for analyzing hydraulic jumps, sequent depths, and energy losses.

In [ ]:
def sequent_depth_rectangular(y1, Q, width):
    """Calculate sequent depth for hydraulic jump in rectangular channel"""
    A1 = width * y1
    V1 = Q / A1
    Fr1 = V1 / math.sqrt(g * y1)
    
    # Analytical solution for rectangular channel
    y2 = (y1 / 2) * (-1 + math.sqrt(1 + 8 * Fr1**2))
    return y2


def sequent_depth_general(channel, y1, Q, initial_guess=None):
    """Calculate sequent depth using momentum equation for general channel shapes"""
    def momentum(y):
        if y <= 0:
            return 0
        A = channel.area(y)
        if A == 0:
            return 0
        y_bar = channel.hydraulic_depth(y) / 2
        M = Q**2 / (g * A) + A * y_bar
        return M
    
    M1 = momentum(y1)
    
    if initial_guess is None:
        A1 = channel.area(y1)
        T1 = channel.top_width(y1)
        if T1 > 0:
            equiv_y1 = A1 / T1
            initial_guess = sequent_depth_rectangular(equiv_y1, Q, T1)
        else:
            initial_guess = y1 * 2
    
    def equation(y2):
        if y2 <= 0:
            return 1e10
        return momentum(y2) - M1
    
    try:
        result = fsolve(equation, initial_guess, full_output=True)
        if result[2] == 1:
            return abs(result[0][0])
    except:
        pass
    
    try:
        if initial_guess > y1:
            y2 = brentq(equation, y1 * 1.01, y1 * 10)
        else:
            y2 = brentq(equation, 0.001, y1 * 0.99)
        return y2
    except:
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def sequent_depth(channel, y1, Q, initial_guess=None):
    """Calculate sequent depth (automatically selects appropriate method)"""
    if isinstance(channel, RectangularChannel):
        return sequent_depth_rectangular(y1, Q, channel.width)
    else:
        return sequent_depth_general(channel, y1, Q, initial_guess)


def energy_loss_jump(channel, y1, y2, Q):
    """Calculate energy loss in hydraulic jump: ΔE = E1 - E2"""
    A1 = channel.area(y1)
    A2 = channel.area(y2)
    
    V1 = Q / A1
    V2 = Q / A2
    
    E1 = y1 + V1**2 / (2 * g)
    E2 = y2 + V2**2 / (2 * g)
    
    return E1 - E2


def energy_loss_fraction(channel, y1, y2, Q):
    """Calculate fraction of energy dissipated: f = ΔE / E1"""
    A1 = channel.area(y1)
    V1 = Q / A1
    E1 = y1 + V1**2 / (2 * g)
    
    delta_E = energy_loss_jump(channel, y1, y2, Q)
    
    return delta_E / E1


def force_on_obstacle(channel, y1, y2, Q):
    """Calculate force on obstacle causing hydraulic jump"""
    rho = 1000  # Water density (kg/m³)
    
    A1 = channel.area(y1)
    A2 = channel.area(y2)
    
    V1 = Q / A1
    V2 = Q / A2
    
    momentum_change = rho * Q * (V1 - V2)
    
    if isinstance(channel, RectangularChannel):
        pressure_change = rho * g * (A1 * y1/2 - A2 * y2/2)
    else:
        D1 = channel.hydraulic_depth(y1)
        D2 = channel.hydraulic_depth(y2)
        pressure_change = rho * g * (A1 * D1/2 - A2 * D2/2)
    
    return momentum_change + pressure_change


print("Hydraulic jump functions loaded successfully!")

## 6. Gradually Varied Flow (GVF)

Functions for solving water surface profiles and GVF problems.

In [ ]:
def gvf_derivative_manning(channel, y, Q, S0, n):
    """Calculate dy/dx for GVF using Manning's equation"""
    if y <= 0:
        return 0
    
    A = channel.area(y)
    R = channel.hydraulic_radius(y)
    T = channel.top_width(y)
    
    if A == 0 or T == 0:
        return 0
    
    V = Q / A
    
    # Friction slope
    Sf = (n * V / (R ** (2/3))) ** 2
    
    # Froude number
    D = channel.hydraulic_depth(y)
    if D <= 0:
        Fr = float('inf')
    else:
        Fr = V / math.sqrt(g * D)
    
    # GVF equation: dy/dx = (S0 - Sf) / (1 - Fr²)
    numerator = S0 - Sf
    denominator = 1 - Fr**2
    
    if abs(denominator) < 1e-10:
        return 0
    
    return numerator / denominator


def solve_gvf_profile_manning(channel, Q, S0, n, y_start, distance, num_steps, direction='downstream'):
    """Solve water surface profile using Euler method"""
    dx = distance / num_steps
    
    if direction == 'upstream':
        dx = -dx
    
    x = np.zeros(num_steps + 1)
    y = np.zeros(num_steps + 1)
    
    x[0] = 0
    y[0] = y_start
    
    for i in range(num_steps):
        dy_dx = gvf_derivative_manning(channel, y[i], Q, S0, n)
        y[i+1] = y[i] + dy_dx * dx
        x[i+1] = x[i] + dx
        
        if y[i+1] < 0.001:
            y[i+1] = 0.001
    
    return x, y


def distance_to_depth_manning(channel, Q, S0, n, y_start, y_target, num_steps=100, max_distance=10000):
    """Calculate distance from y_start to y_target using GVF"""
    if abs(y_start - y_target) < 0.001:
        return 0
    
    direction = 'upstream' if y_target > y_start else 'downstream'
    distance = abs(y_target - y_start) * 100
    
    for iteration in range(10):
        x, y = solve_gvf_profile_manning(channel, Q, S0, n, y_start, distance, num_steps, direction)
        y_end = y[-1]
        
        if abs(y_end - y_target) < 0.01:
            return abs(x[-1])
        
        if direction == 'upstream':
            distance = distance * 1.5 if y_end < y_target else distance * 0.7
        else:
            distance = distance * 1.5 if y_end > y_target else distance * 0.7
        
        if distance > max_distance:
            distance = max_distance
            break
    
    return abs(x[-1])


def classify_gvf_curve(S0, S_c, y, y_n, y_c):
    """Classify GVF curve type (M1, M2, S1, S2, etc.)"""
    # Slope type
    if abs(S0) < 1e-10:
        slope_type = 'H'
    elif S0 < 0:
        slope_type = 'A'
    elif abs(S0 - S_c) < S_c * 0.01:
        slope_type = 'C'
    elif S0 < S_c:
        slope_type = 'M'
    else:
        slope_type = 'S'
    
    # Zone determination
    if slope_type == 'M':
        if y > y_n:
            zone = '1'
        elif y > y_c:
            zone = '2'
        else:
            zone = '3'
    elif slope_type == 'S':
        if y > y_c:
            zone = '1'
        elif y > y_n:
            zone = '2'
        else:
            zone = '3'
    elif slope_type == 'C':
        zone = '1' if y > y_c else '3'
    elif slope_type in ['H', 'A']:
        zone = '2' if y > y_c else '3'
    else:
        zone = '?'
    
    return slope_type + zone


print("GVF functions loaded successfully!")

## 7. Weir and Sluice Gate Analysis

Functions for analyzing flow over weirs and under sluice gates.

In [ ]:
def weir_upstream_depth(channel, Q, weir_height, normal_depth=None):
    """Calculate upstream depth for flow over a broad-crested weir"""
    y_c = critical_depth(channel, Q)
    E_c = (3/2) * y_c
    E_upstream = E_c + weir_height
    
    def equation(y):
        if y <= weir_height:
            return 1e10
        E = specific_energy(channel, y, Q)
        return E - E_upstream
    
    try:
        y_upstream = brentq(equation, weir_height + y_c, E_upstream * 2)
        return y_upstream
    except:
        initial_guess = weir_height + y_c * 1.5
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def weir_depth_over_crest(channel, Q, weir_height, E_upstream):
    """Calculate depth over the crest of a broad-crested weir"""
    E_crest = E_upstream - weir_height
    y_c = critical_depth(channel, Q)
    
    def equation(y):
        if y <= 0:
            return 1e10
        E = specific_energy(channel, y, Q)
        return E - E_crest
    
    try:
        y_over_crest = brentq(equation, 0.001, E_crest)
        return y_over_crest
    except:
        result = fsolve(equation, y_c)
        return abs(result[0])


def sluice_gate_discharge(channel, y_upstream, y_downstream, energy_loss_coeff=0.0):
    """Calculate discharge under a sluice gate using Bernoulli equation"""
    A_down = channel.area(y_downstream)
    
    if y_upstream <= y_downstream:
        return 0
    
    V2 = math.sqrt(2 * g * (y_upstream - y_downstream) / (1 + energy_loss_coeff))
    Q = A_down * V2
    
    return Q


def sluice_gate_upstream_depth(channel, Q, y_downstream):
    """Calculate upstream depth for given discharge and downstream depth"""
    E2 = specific_energy(channel, y_downstream, Q)
    
    def equation(y):
        if y <= y_downstream:
            return 1e10
        E1 = specific_energy(channel, y, Q)
        return E1 - E2
    
    try:
        y_c = critical_depth(channel, Q)
        y_upstream = brentq(equation, max(y_c, y_downstream), E2 * 2)
        return y_upstream
    except:
        initial_guess = y_downstream * 3
        result = fsolve(equation, initial_guess)
        return abs(result[0])


def sluice_gate_force(channel, y_upstream, y_downstream, Q):
    """Calculate force on sluice gate using momentum equation"""
    rho = 1000  # Water density (kg/m³)
    
    A1 = channel.area(y_upstream)
    A2 = channel.area(y_downstream)
    
    V1 = Q / A1 if A1 > 0 else 0
    V2 = Q / A2 if A2 > 0 else 0
    
    momentum_change = rho * Q * (V1 - V2)
    
    D1 = channel.hydraulic_depth(y_upstream)
    D2 = channel.hydraulic_depth(y_downstream)
    
    pressure_change = rho * g * (A1 * D1/2 - A2 * D2/2)
    
    return momentum_change + pressure_change


def free_overfall_depth(channel, Q):
    """Calculate depth at a free overfall (approximately 71.5% of critical depth)"""
    y_c = critical_depth(channel, Q)
    return 0.715 * y_c


print("Weir and sluice gate functions loaded successfully!")

## 8. High-Level Solver Class

A comprehensive solver class that provides a unified interface for all types of open channel flow problems.

In [ ]:
class OpenChannelSolver:
    """Main solver class for open channel flow problems"""
    
    def __init__(self):
        self.g = 9.81
    
    def solve_basic_flow_problem(self, channel_type, channel_params, Q, S, n=None, C=None):
        """Solve basic flow problem: normal depth, critical depth, Froude number, etc."""
        channel = self._create_channel(channel_type, channel_params)
        
        results = {
            'channel_type': channel_type,
            'discharge': Q
        }
        
        # Critical depth
        y_c = critical_depth(channel, Q)
        results['critical_depth'] = y_c
        
        # Process slopes
        if not isinstance(S, list):
            S = [S]
        
        for i, slope in enumerate(S):
            slope_results = {}
            
            if n is not None:
                y_n = normal_depth_manning(channel, Q, slope, n)
                slope_results['normal_depth'] = y_n
                
                Fr_n = froude_number(channel, y_n, Q)
                slope_results['froude_number'] = Fr_n
                slope_results['flow_regime'] = flow_regime(Fr_n)
            
            results[f'slope_{i+1}'] = slope_results
        
        # Critical slope
        if n is not None:
            S_c = critical_slope_manning(channel, Q, n, y_c)
            results['critical_slope'] = S_c
        
        return results
    
    def solve_hydraulic_jump_problem(self, channel_type, channel_params, Q, y1=None, y2=None):
        """Solve hydraulic jump problems"""
        channel = self._create_channel(channel_type, channel_params)
        
        results = {'discharge': Q}
        
        if y1 is not None:
            results['depth_1'] = y1
            Fr1 = froude_number(channel, y1, Q)
            results['froude_1'] = Fr1
            
            y2_calc = sequent_depth(channel, y1, Q)
            results['sequent_depth'] = y2_calc
            
            dE = energy_loss_jump(channel, y1, y2_calc, Q)
            frac = energy_loss_fraction(channel, y1, y2_calc, Q)
            results['energy_loss'] = dE
            results['energy_loss_fraction'] = frac
        
        return results
    
    def solve_gvf_problem(self, channel_type, channel_params, Q, S, n, y_start, y_target, num_steps=100):
        """Solve gradually varied flow problem"""
        channel = self._create_channel(channel_type, channel_params)
        
        results = {
            'discharge': Q,
            'slope': S,
            'manning_n': n,
            'start_depth': y_start,
            'target_depth': y_target
        }
        
        y_n = normal_depth_manning(channel, Q, S, n)
        y_c = critical_depth(channel, Q)
        S_c = critical_slope_manning(channel, Q, n, y_c)
        
        results['normal_depth'] = y_n
        results['critical_depth'] = y_c
        results['critical_slope'] = S_c
        results['slope_classification'] = slope_classification(S, S_c)
        
        gvf_curve = classify_gvf_curve(S, S_c, y_start, y_n, y_c)
        results['gvf_curve'] = gvf_curve
        
        distance = distance_to_depth_manning(channel, Q, S, n, y_start, y_target, num_steps)
        results['distance'] = distance
        
        return results
    
    def _create_channel(self, channel_type, params):
        """Factory method to create channel object"""
        if channel_type == 'rectangular':
            return RectangularChannel(params['width'])
        elif channel_type == 'trapezoidal':
            return TrapezoidalChannel(params['bottom_width'], params['side_slope'])
        elif channel_type == 'circular':
            return CircularChannel(params['diameter'])
        elif channel_type == 'triangular':
            if 'side_slope' in params:
                return TriangularChannel(side_slope=params['side_slope'])
            else:
                return TriangularChannel(semi_angle=params['semi_angle'])
        elif channel_type == 'wide':
            return WideChannel()
        else:
            raise ValueError(f"Unknown channel type: {channel_type}")
    
    def print_results(self, results, title="Results"):
        """Pretty print results"""
        print("\n" + "="*70)
        print(title)
        print("="*70)
        
        for key, value in results.items():
            if isinstance(value, dict):
                print(f"\n{key}:")
                for k, v in value.items():
                    if isinstance(v, (int, float)):
                        print(f"  {k}: {v:.4f}")
                    else:
                        print(f"  {k}: {v}")
            elif isinstance(value, (int, float)):
                print(f"{key}: {value:.4f}")
            else:
                print(f"{key}: {value}")


print("OpenChannelSolver class loaded successfully!")
print("\n" + "="*70)
print("ALL MODULES LOADED - READY TO SOLVE PROBLEMS!")
print("="*70)

## 9. Example Problems

Here are several worked examples demonstrating how to use the solver.

### Example 1: Rectangular Channel - Normal and Critical Depths

In [ ]:
# Create solver
solver = OpenChannelSolver()

# Solve for rectangular channel with width = 5m, Q = 20 m³/s, n = 0.02
results = solver.solve_basic_flow_problem(
    channel_type='rectangular',
    channel_params={'width': 5.0},
    Q=20.0,
    S=[0.001, 0.01],  # Two different slopes
    n=0.02
)

solver.print_results(results, "Example 1: Rectangular Channel")

### Example 2: Trapezoidal Channel

In [ ]:
# Trapezoidal channel: bottom width = 0.6m, side slope = 0.75, Q = 2.6 m³/s
results = solver.solve_basic_flow_problem(
    channel_type='trapezoidal',
    channel_params={'bottom_width': 0.6, 'side_slope': 0.75},
    Q=2.6,
    S=1/2500,
    n=0.012
)

solver.print_results(results, "Example 2: Trapezoidal Channel")

### Example 3: Circular Channel (Pipe)

In [ ]:
# Circular channel: diameter = 1.4m (radius = 0.7m), Q = 0.8 m³/s
results = solver.solve_basic_flow_problem(
    channel_type='circular',
    channel_params={'diameter': 1.4},
    Q=0.8,
    S=0.02,  # 2% slope
    n=0.013
)

solver.print_results(results, "Example 3: Circular Channel")

### Example 4: Hydraulic Jump in Rectangular Channel

In [ ]:
# Rectangular channel: width = 3m, upstream depth = 0.3m after sluice gate
channel = RectangularChannel(width=3.0)
Q = 13.0  # m³/s
y1 = 0.3  # m

# Calculate sequent depth
y2 = sequent_depth(channel, y1, Q)
print(f"Upstream depth (supercritical): y1 = {y1:.3f} m")
print(f"Sequent depth (subcritical): y2 = {y2:.3f} m")

# Calculate energy loss
dE = energy_loss_jump(channel, y1, y2, Q)
frac = energy_loss_fraction(channel, y1, y2, Q)
print(f"\nEnergy loss in jump: ΔE = {dE:.3f} m")
print(f"Energy loss fraction: {frac*100:.1f}%")

# Froude numbers
Fr1 = froude_number(channel, y1, Q)
Fr2 = froude_number(channel, y2, Q)
print(f"\nFroude number upstream: Fr1 = {Fr1:.3f} ({flow_regime(Fr1)})")
print(f"Froude number downstream: Fr2 = {Fr2:.3f} ({flow_regime(Fr2)})")

### Example 5: Sluice Gate Problem

In [ ]:
# Sluice gate in rectangular channel
channel = RectangularChannel(width=3.0)
y_upstream = 2.0  # m
y_downstream_gate = 0.3  # m (depth just after gate)

# Calculate discharge
Q = sluice_gate_discharge(channel, y_upstream, y_downstream_gate)
print(f"Upstream depth: {y_upstream} m")
print(f"Downstream depth (at gate): {y_downstream_gate} m")
print(f"Discharge: Q = {Q:.3f} m³/s")

# Normal depth downstream (given slope and n)
S = 1/1000
n = 0.014
y_n = normal_depth_manning(channel, Q, S, n)
print(f"\nNormal depth downstream: y_n = {y_n:.3f} m")

# Force on gate
F = sluice_gate_force(channel, y_upstream, y_downstream_gate, Q)
print(f"Force on gate: F = {F/1000:.1f} kN")

### Example 6: Gradually Varied Flow (GVF) - Free Overfall

In [ ]:
# Wide channel with free overfall
results = solver.solve_gvf_problem(
    channel_type='wide',
    channel_params={},
    Q=0.5,  # m³/s per m width
    S=2e-5,
    n=0.01,
    y_start=0.715 * critical_depth(WideChannel(), 0.5),  # Depth at overfall
    y_target=1.0,  # Target depth upstream
    num_steps=100
)

solver.print_results(results, "Example 6: GVF - Free Overfall")

### Example 7: Broad-Crested Weir

In [ ]:
# Wide channel with broad-crested weir
channel = WideChannel()
Q = 1.5  # m³/s per m width
S = 2e-4
n = 0.015
weir_height = 0.2  # m

# Normal and critical depths
y_n = normal_depth_manning(channel, Q, S, n)
y_c = critical_depth(channel, Q)

print(f"Normal depth: y_n = {y_n:.3f} m")
print(f"Critical depth: y_c = {y_c:.3f} m")

# Upstream depth for critical flow over weir
y_upstream = weir_upstream_depth(channel, Q, weir_height)
print(f"\nWeir height: {weir_height} m")
print(f"Upstream depth: {y_upstream:.3f} m")
print(f"Depth over weir crest: {y_c:.3f} m (critical depth)")
print(f"Downstream depth: {y_n:.3f} m (returns to normal)")

### Example 8: Triangular Channel with Hydraulic Jump

In [ ]:
# Triangular channel with semi-angle = 40°
results = solver.solve_hydraulic_jump_problem(
    channel_type='triangular',
    channel_params={'semi_angle': 40},
    Q=16.0,
    y1=1.85
)

solver.print_results(results, "Example 8: Triangular Channel - Hydraulic Jump")

## 10. Custom Problem Solver

Use this cell to solve your own custom problems:

In [ ]:
# YOUR CUSTOM PROBLEM HERE
# Example: Solve for a rectangular channel

# Create channel
channel = RectangularChannel(width=4.0)  # 4m wide

# Given parameters
Q = 15.0  # Discharge (m³/s)
S = 0.005  # Slope
n = 0.015  # Manning's n

# Calculate normal depth
y_n = normal_depth_manning(channel, Q, S, n)
print(f"Normal depth: {y_n:.3f} m")

# Calculate critical depth
y_c = critical_depth(channel, Q)
print(f"Critical depth: {y_c:.3f} m")

# Calculate Froude number at normal depth
Fr = froude_number(channel, y_n, Q)
print(f"Froude number: {Fr:.3f}")
print(f"Flow regime: {flow_regime(Fr)}")

# Calculate specific energy
E = specific_energy(channel, y_n, Q)
print(f"Specific energy: {E:.3f} m")

## Summary

This notebook provides a complete open channel flow calculator with:

1. **Channel Geometries**: Rectangular, Trapezoidal, Circular, Triangular, Wide
2. **Flow Calculations**: Normal depth, Critical depth, Froude number, Specific energy
3. **Hydraulic Jumps**: Sequent depth, Energy loss, Force calculations
4. **GVF Analysis**: Water surface profiles, Distance calculations, Curve classification
5. **Hydraulic Structures**: Weirs, Sluice gates, Free overfalls

**To use this notebook:**
1. Run all cells in order (Cell → Run All)
2. Modify the example problems or create your own in the custom problem cell
3. All functions return numerical values that you can use in your calculations

**Need help?** Check the function docstrings or refer to the examples above.

---

**Developed by:** Open Channel Flow Calculator Team  
**For:** Hydraulic Engineering Applications  
**License:** Educational Use

---